In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
from model_input_old import input_pipeline
from model import CNNModel, RNNModel
import pickle

In [2]:
config = {}
# Get from dataset.
config['num_test_samples'] = 2174
config['lr'] = {}
config['lr']['batch_size'] = 16
config['batch_size'] = 16

config['num_epochs'] = 1
config['model_dir'] = '../runs/lr=(False_0.97_exponential_1000_0.001_16)_cnn=(0.5_(5x3x3x3)_(16x32x64x128)_(512x256xNone))_rnn=(20_average_1_256_16)_1497524781'
config['checkpoint_id'] = None # If None, the last checkpoint will be used.

config['img_height'] = 80
config['img_width'] = 80
config['img_num_channels'] = 3
config['skeleton_size'] = 180

# CNN model parameters
config['cnn'] = pickle.load(open(os.path.join(config['model_dir'], "config_cnn.pkl"), 'rb'))

# RNN model parameters
config['rnn'] = pickle.load(open(os.path.join(config['model_dir'], "config_rnn.pkl"), 'rb'))

config['ip_queue_capacity'] = config['batch_size']*50
config['ip_num_read_threads'] = 1

config['test_data_dir'] = "../data/test/"
config['test_file_format'] = "dataTest_%d.tfrecords"
config['test_file_ids'] = list(range(1,16))

In [3]:
# Create a list of tfRecord input files.
test_filenames = [os.path.join(config['test_data_dir'], config['test_file_format'] % i) for i in config['test_file_ids']]
# Create data loading operators. This will be represented as a node in the computational graph.
test_batch_samples_op, test_batch_ids_op, test_batch_seq_len_op = input_pipeline(test_filenames, config, name='test_input_pipeline', shuffle=False, mode="inference")

sess = tf.Session()
init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
sess.run(init_op)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

# Test graph.
with tf.name_scope("Inference"):
    # Create model
    inferCnnModel = CNNModel(config=config['cnn'],
                            input_op=test_batch_samples_op, 
                            mode='inference')
    
    infer_cnn_representations = inferCnnModel.build_graph()
    
    inferModel = RNNModel(config=config['rnn'], 
                            input_op=infer_cnn_representations, 
                            target_op=None, 
                            seq_len_op=test_batch_seq_len_op,
                          
                            mode="inference")
    inferModel.build_graph()
    
# Restore computation graph.
saver = tf.train.Saver()
# Restore variables.
checkpoint_path = config['checkpoint_id']
if checkpoint_path is None:
    checkpoint_path = tf.train.latest_checkpoint(config['model_dir'])
print("Evaluating " + checkpoint_path)
saver.restore(sess, checkpoint_path)


Evaluating /home/martipa/project/runs/lr=(False_0.97_exponential_1000_0.001_16)_cnn=(0.5_(5x3x3x3)_(16x32x64x128)_(512x256xNone))_rnn=(20_average_1_256_16)_1497524781/model-3570


In [ ]:
# # Create a list of tfRecord input files.
# test_filenames = [os.path.join(config['test_data_dir'], config['test_file_format'] % i) for i in config['test_file_ids']]
# # Create data loading operators. This will be represented as a node in the computational graph.
# test_batch_samples_op, test_batch_ids_op, test_batch_seq_len_op = input_pipeline(test_filenames, config, name='test_input_pipeline', shuffle=False, mode="inference")

# sess = tf.Session()
# init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
# sess.run(init_op)

# coord = tf.train.Coordinator()
# threads = tf.train.start_queue_runners(sess=sess, coord=coord)

# # Test graph.
# with tf.name_scope("Inference"):
#     # Create model
#     inferCnnModel = CNNModel(config=config['cnn'],
#                             input_op=test_batch_samples_op, 
#                             mode='inference')
    
#     infer_cnn_representations = inferCnnModel.build_graph()
    
#     inferModel = RNNModel(config=config['rnn'], 
#                             input_op=infer_cnn_representations, 
#                             target_op=None, 
#                             seq_len_op=test_batch_seq_len_op,
#                             mode="inference")
    
#     inferModel.build_graph()
    
# # Restore computation graph.
# saver = tf.train.Saver()

# # restore variables

# if config['checkpoint_id'] is None:
#     checkpoint_path = tf.train.latest_checkpoint(config['model_dir'])
# else:
#     checkpoint_path = os.path.abspath(os.path.join(config['model_dir'], config['checkpoint_id']))
    
# print("Evaluating " + checkpoint_path)
# saver.restore(sess, checkpoint_path)

# print('\ndone')

In [4]:
# Evaluation loop
test_predictions = []
test_sample_ids = []
try:
    while not coord.should_stop():
        # Get predicted labels and sample ids for submission csv.
        [predictions, sample_ids] = sess.run([inferModel.predictions, test_batch_ids_op], feed_dict={})
        test_predictions.extend(predictions)
        test_sample_ids.extend(sample_ids)

except tf.errors.OutOfRangeError:
    print('Done.')
finally:
    # When done, ask the threads to stop.
    coord.request_stop()   

# Wait for threads to finish.
coord.join(threads)

# Now you have your predictions. Do whatever you want:

Done.


In [ ]:
print(test_predictions[:100])
print(test_sample_ids[:100])

In [5]:
final_predictions = [x+1 for x in test_predictions]

submission = np.zeros((config['num_test_samples'], 2))

for i in range(0, config['num_test_samples']):
    submission[i, 0] = i + 1
    submission[i, 1] = final_predictions[i]
    
print(submission.shape)
    
# should add checkpoint to name
submission_name = '../submissions/submission_{}.csv'.format("1")

with open(submission_name, 'wb') as f:
    f.write(b"Id,Prediction\n")
    np.savetxt(f,submission,fmt='%i', delimiter=',')
    
print('done')

(2174, 2)
done


In [ ]:
# test_predictions = np.zeros((0,))
# test_sample_ids = np.zeros((0,))

# try:
#     while not coord.should_stop():
#         # Get predicted labels and sample ids for submission csv.
#         [predictions, sample_ids] = sess.run([inferModel.predictions, test_batch_ids_op], feed_dict={})
#         test_predictions = np.concatenate((test_predictions, predictions))
#         test_sample_ids = np.concatenate((test_sample_ids, sample_ids))

# except tf.errors.OutOfRangeError:
#     print('done')
# finally:
#     # When done, ask the threads to stop.
#     coord.request_stop()   

# # Wait for threads to finish.
# coord.join(threads)

In [ ]:
test_predictions += 1
submission = np.stack((test_prediction, test_sample_ids), axis=1)

# unclean, checkpoint_id possibly None, but oh lord...
submission_name = '../submission_{}'.format(config['model_dir'])

with open(submission_name, 'wb') as f:
    f.write(b"Id,Prediction\n")
    np.savetxt(f,submission,fmt='%i', delimiter=',')
    
print('done')